In [ ]:
import requests
import ssl
import pandas as pd
from bs4 import BeautifulSoup
import re
import os

In [ ]:
#### 리뷰 한권씩 추출
def bookRV(num):
    url = "https://product.kyobobook.co.kr/api/review/list?page={}&pageLimit=10&reviewSort=001&revwPatrCode=000&saleCmdtid=S000202652825"  #해당 도서 상세 페이지 url
    response = requests.get(url.format(num), headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)

    if response.status_code == 200:
        try:
            data = response.json()
            review_list = data['data']['reviewList']

            # 필요한 정보를 추출하여 리스트로 정리
            formatted_reviews = []
            for review in review_list:
                formatted_review = {
                    'saleCmdtid': review['saleCmdtid'],
                    'cmdtName': review['cmdtName'],
                    'revwNum': review['revwNum'],
                    'revwCntt': review['revwCntt'],
                    'revwRvgr': review['revwRvgr'],
                    'reviewRecommendCount': review['reviewRecommendCount']
                }
                formatted_reviews.append(formatted_review)

            return formatted_reviews
    else:
        print(f"Failed to retrieve the page for page {num}.")
        return []

book_rvlist = []
for num in range(1, 11):
    book_rvlist.extend(bookRV(num))

# 데이터 프레임으로 전환
df_review = pd.DataFrame(book_rvlist) 
df_review.to_csv("쇼펜하우어아포리즘:당신의인생이왜힘들지않아야한다고생각하십니까S000202652825.csv", encoding='utf-8-sig', index=False)  # CSV저장
df_review

In [ ]:
#### 전체 리뷰 추출
#soup = BeautifulSoup(html_content, 'html.parser')
data_id_list = []

# saleCmdtid가 들어있는 CSV 파일 읽고 데이터 프레임으로
csv_file_path = r"2020best.csv"  # CSV 파일 경로
data_id_df = pd.read_csv(csv_file_path)  # CSV 파일을 DataFrame으로 읽기

# 'data_id' 열의 값을 리스트로 가져오기
data_id_list = data_id_df['SCID'].tolist()

def bookRV(num, data_id):
    ssl_ctx = ssl.create_default_context()
    ssl_ctx.set_ciphers('DEFAULT@SECLEVEL=1')

    # num=리뷰페이지 data_id위에서 만든 리스트에서 가져올 것
    url = f"https://product.kyobobook.co.kr/api/review/list?page={num}&pageLimit=10&reviewSort=001&revwPatrCode=000&saleCmdtid={data_id}"
    response = requests.get(url, verify=True, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)

    if response.status_code == 200:
        data = response.json()
        review_list = data['data']['reviewList']

        # 필요한 정보를 추출하여 리스트로 정리 후 추출
        formatted_reviews = []
        for review in review_list:
            formatted_review = {
                'saleCmdtid': review['saleCmdtid'],
                'cmdtName': review['cmdtName'],
                'revwCntt': review['revwCntt'],
                'revwRvgr': review['revwRvgr'],
                'reviewRecommendCount': review['reviewRecommendCount'],
                'revwNum' : review['reviewRecommendCount']
            }
            formatted_reviews.append(formatted_review)
        return formatted_reviews
        
    else:  # 실패시
        print(f"Failed to retrieve the page for page {num}.")
        return []

for data_id in data_id_list:   # data_id순환
    book_rvlist = []    # 리뷰 쌓을 리스트
    cmdtName = None  # cmdtName 변수 초기화
    for num in range(1, 10):   # 리뷰 페이지 순환
        if not cmdtName:
            cmdtName = bookRV(1, data_id)[0]['cmdtName']  # 첫 번째 페이지의 cmdtName 가져오기
        book_rvlist.extend(bookRV(num, data_id))
        
    # 파일명에 있는 공백을 언더스코어로 대체하여 저장
    cmdtName = re.sub(r'[^\w\s]', '', cmdtName).replace(" ", "_")
    df_review = pd.DataFrame(book_rvlist)
    df_review.to_csv(f"last2020\\{cmdtName}_{data_id}.csv", encoding='utf-8-sig', index=False)
    
# 작업 끝나면 clear출력
print("clear")

In [ ]:
#### 폴더에 들어있는 다량의 csv 파일 합치기
folder_path = 'last2019' # 합칠 csv파일들이 모여 있는 경로
dfs = []
# 폴더 내의 모든 파일에 대해 반복
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):  # 확장자가 .csv인 파일만 처리
        file_path = os.path.join(folder_path, file_name)
        # CSV 파일을 DataFrame으로 읽기
        df = pd.read_csv(file_path)
        # 필요한 컬럼만 선택
        df = df[['saleCmdtid','cmdtName','revwCntt','revwNum','revwRvgr','reviewRecommendCount']]
        # DataFrame을 리스트에 추가
        dfs.append(df)
# 리스트에 저장된 모든 DataFrame을 하나로 합치기
combined_df = pd.concat(dfs, ignore_index=True)
# 모든 데이터를 담은 하나의 CSV 파일로 저장
combined_df.to_csv("last2019\reviewRecommendCount.csv", encoding='utf-8', index=False)

In [ ]:
#### 리뷰 평점 기준 4점 만점에서 10점 만점으로 변경
original_file_path = "ple" # 변경하기 전 경로
output_file_path = "ple_all.csv" # 변경 후 저장될 졍로
# CSV 파일을 읽어오기
df = pd.read_csv(original_file_path, encoding='utf-8') #utf8일 경우 encoding 필요 없음

#  'revwRvgr' 컬럼에 2.5곱하기
df['revwRvgr'] = df['revwRvgr'] * 2.5

# 만약 'revwRvgr' 컬럼이 정수형이라면, 데이터 타입을 다시 정수형으로 변환
df['revwRvgr'] = df['revwRvgr'].astype(int)

# 변경된 CSV 파일 저장하기
df.to_csv(output_file_path, encoding='utf-8', index=False) #utf8일 경우 encoding 필요 없음

In [ ]:
#### 중복 삭제 후 저장
# CSV 파일을 읽어와 데이터프레임으로 변환
df = pd.read_csv("k_review_utf8.csv")
df.shape
# 중복된 행 중 첫 번째 행만 남기고 나머지는 삭제
df.drop_duplicates(keep='first', inplace=True)

# 변경된 데이터프레임을 새로운 CSV 파일로 저장
df.to_csv("k_review_finish_utf8.csv", index=False)

In [ ]:
#### 컬럼 추출 후 합치기

# 첫 번째 CSV 파일 경로
file1_path = 'file1.csv'

# 두 번째 CSV 파일 경로
file2_path = 'K_Book_Reveiw.csv'

# 첫 번째 CSV 파일 읽기
# 'title'과 함께 필요한 컬럼만 선택하여 읽기
df1 = pd.read_csv(file1_path, usecols=['saleCmdtid', 'ISBN'])

# 두 번째 CSV 파일 읽기
df2 = pd.read_csv(file2_path)

# 두 데이터프레임을 공통 컬럼 'title'을 기준으로 병합 (inner 조인)
merged_df = pd.merge(df1, df2, on='saleCmdtid', how='inner')

# 변경된 CSV 파일 저장하기
output_file_path = 'K_Book_Reveiw_utf8_modified_ISBN.csv'
# ANSI로 인코딩
merged_df.to_csv(output_file_path, encoding='utf-8', index=False) 
# # utf-8로 인코딩
# merged_df.to_csv(output_file_path, index=False)